In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation,prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Here're just a preparatory phase for prover
- Create dummy data associated with their dataset for verifier to use for generating onnx model.
- Generate data commitment for their dataset.

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [3]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

## Step 2
- User (Verifier) defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `computation_to_model()` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`, while also export `precal_witness_arr.json` which is the constant witness needed for verifier to generate the same onnx.

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    # print('x comm: ', x)
    return s.mean(x)

error = 0.01


# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

==== Generate & Calibrate Setting ====
scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":1,"lookup_range":[-288,300],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":3936,"total_assignments":1129,"total_const_size":466,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3],"module_sizes":{"kzg":[],"poseidon":[3936,[1]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-4,4]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715154595000}


## Step 3
- User (Verifier) gets `precal_witness_arr.json`, and generate the same onnx format with same parameter.

In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False,error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

## Step 4
- Verifier sets up vk, pk.
- Prover generates proof with `prover_gen_proof`


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.5352780818939209 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 42.125
==== Generating Proof ====
proof:  {'instances': [['11e5950d0c875140b38d8b4bc0997697b7b183cfdbc19e767d87caf0020da12a', '0100000000000000000000000000000000000000000000000000000000000000', '5101000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x25b0832442641ed7435618e958432c891a1eded25dcbc39fdc2ca7f38a17694a1b0ad1521d750fa24677a704f6d3210d37fa42fc14035ff7908c19862a89386707062793a27ea4d2480137bc01d708ea8bf1152aa9eb8d910fb33b652f1dd2a605c3126f907e844ac250fb66e7297cfb5033e53d65f00554532aa12b8d23ba5a0a5497784ef4600b67260c9fbfaaef2744b7c979a7b09992c9ece175746b1d000a1fdcb2bf26f0e5f996ece24bcbd97f459703257878185f136d8587073b087b1e022559dc93ffcf7f6ce74aecbec8ab5611380373e27458a8d3ebc4c01652950abcd0d88b81f5635b16a9440ac884b20231bbaaf9fae54fcfa0d23fcb86453010dee3ddd6b60f4b0ead94acf52046056283c5a8b21166c3d7a48b79a09cd29905b02e3f35592

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [42.125]
